Readme 

In [ ]:
#Import relevant libraries ----- copied from other ------ consider which ones to keep 

###watch week 5 Do 

import pandas as pd
import numpy as np
# for Box-Cox Transformation
from scipy import stats

# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

# plotting modules
import seaborn as sns
import missingno

import matplotlib.pyplot as plt 

#string to numeric 
from sklearn.preprocessing import LabelEncoder  

In [ ]:
#Read file and show overview of data 
titanic_df = pd.read_csv("Titanic.csv") 
titanic_df.head() 

#### Explanation of the column names 

An explanation of the column names is available on kaggle (https://www.kaggle.com/c/titanic/data):  

- Each passenger gets a PassengerId. 
- Survived describes if they survived the distater or not (0 = No, 1 = Yes).  
- Pclass refers to the ticket class each passenger purchased/stayed in and is divided into 1st (1), 2nd (2) and 3rd (3) class. This can be considered representative of the socio-economic status of the passenger. 
- The name of each passenger is recorded. 
- Sex is either male or female.  
- Age is recorded in years. 
- SibSp refers to the number of siblings or if the passenger had a spouse on board of the titanic. 
- Parch is short for parent/child i.e. the number of parents or children the passenger had on board (some children only travelled with a nanny leading to some children having a value of 0 for Parch). 
- Ticket contains the ticket number the passenger purchased. 
- Fare is how much the passenger paid. 
- Cabin is the cabin number the passenger stayed in. 
- Embarked is the initial of the harbour the passenger embarked at (C=Cherbourg, Q=Queenstown, S=Southampton).  

In [ ]:
#Looking at the data 
titanic_df.shape
titanic_df.info() 

In [ ]:
#Cleaning the data: check for missing entries  
missing_entries = titanic_df.isnull().sum() 
missing_entries 

In [ ]:
#Graph of missing values
missingno.matrix(titanic_df)

At 687 out of 891, a signifcant amount of entries for Cabin are missing. There are also 177 missing entries for Age, while for Embarked only 2 are missing. 

As so many entries for Cabin are missing and the layout of the Titanic in relation to each cabin is not given (as the position of the cabin might have affected how easy it was to get to the lifeboats), the column will be dropped entirely. 

Many entries for Age are missing, however being older or younger might affect your chances of survival so it is an important column to keep. Removing these entries would also affect the other columns. The missing entries will be filled with the mean of the Age column as age in a continuous variable.   

For Embarked only two entries are missing. These can be filled in using the mode of the column as it is a categorical variable.

Additionally, it is important to consider the other columns. It is unlikely that the PassengerId, Ticket or Name had a huge impact on survival in general so they too will be dropped. 

In [ ]:
#Missing entries - dropping the Cabin, PassengerId, Name and Ticket column 
titanic_df.drop(['Cabin'], inplace=True, axis=1) 
titanic_df.drop(['PassengerId'], inplace=True, axis=1) 
titanic_df.drop(['Name'], inplace=True, axis=1) 
titanic_df.drop(['Ticket'], inplace=True, axis=1) 

titanic_df.head() 

In [ ]:
#Missing entries: Age 
mean_age = titanic_df['Age'].mean() 
titanic_df.loc[titanic_df['Age'].isnull(), 'Age'] = mean_age 

In [ ]:
#Missing entries: Embarked column 
titanic_df['Embarked'].unique()

In [ ]:
#Missing entries: filling Embarked column 
titanic_df['Embarked'].mode() 
titanic_df.loc[titanic_df['Embarked'].isnull(), 'Embarked'] ='S'

titanic_df['Embarked'].unique()

In [ ]:
#Overview of all columns 
titanic_df.hist()
plt.tight_layout()

In [2]:
#m pairplot  

NameError: name 'sns' is not defined

### Most important factor in determining survival on Titanic? 

In [ ]:
#Looking at how many survived the disaster 
sns.countplot(x='Survived', data=titanic_df)

### maybe talk more 
More people died during the distater than survived it.  

In [ ]:
#Looking at how each category relates to survival on its own 
# Pclass vs Survived 

sns.histplot(data=titanic_df, x='Pclass', hue = 'Survived', multiple='stack')

The majority of passengers on the Titanic were 3rd class. Most did not survive the sinking of the ship. In 2nd class roughly half survived. In 1st class more than half survived the distater. Being a 3rd class passenger drastically lowered you chances at survival. 

In [ ]:
# Sex vs Survived 
sns.histplot(data=titanic_df, x='Sex', hue = 'Survived', multiple='stack')

Even though there were only about half as many female passenger as there were male passengers, they had roughly double the chance at surviving the sinking. 

In [ ]:
# Age vs Survived 
sns.histplot(data=titanic_df, x='Age', hue = 'Survived', multiple='stack')

In [ ]:
sns.violinplot(data = titanic_df, x = 'Age', y = 'Survived')
#or sns.violinplot(data = titanic_df, x = 'Pclass', y = 'Age', hue = 'Survived')

At most ages, the chances at survival seem to be somewhat less than 50%. Two stand outs are the survival chances of anyone below 10 years of age and around 30 years of age. More children survived than died. On the other side, if you were around 30 years of age you had a much higher chance of dying. However, one thing to consider here is that the Age column had a lot of missing entries and these were filled in using the mean value of the column, which might have affected the outcome of this graph.  

In [ ]:
# SibSp vs Survived 
sns.histplot(data=titanic_df, x='SibSp', hue = 'Survived', multiple='stack')

Most people that died did not have a spouse or siblings on board. Neither did most people that survived. Having one spouse or sibling meant you were roughly half as likely to survive as not. Only a small number had two or more. Having three or more meant you were more likely to die. This could potentially be because they would try to stay together as a family making it harder to get to the lifeboats. 

In [ ]:
# Parch vs Survived 
sns.histplot(data=titanic_df, x='Parch', hue = 'Survived', multiple='stack')

Same as with the graph for SibSp vs Survival, the majority of people that died and survived did not have a parent or child with them on board. Having one or two on board brought the chance of survival to roughly half. Only a very small number had more than two. 

In [ ]:
# Fare vs Survived 
sns.histplot(data=titanic_df, x='Fare', hue = 'Survived', multiple='stack')

Most people that died had paid a lower fare. People that paid a higher fare were more likely to survive. 

In [ ]:
# Embarked vs Survived 
sns.histplot(data=titanic_df, x='Embarked', hue = 'Survived', multiple='stack')

The majority of passengers embarked in Southampton. Most people that survived and did not survive embarked in Southampton. Comparing these to Cherbourg and Queenstown, it shows that passengers embarking in Cherbourg had a roughly equal chance of survival, but it was less than half if a passenger embarked in Queenstown. 

In [ ]:
#Using a heatmap to look at the correlation between each variable 

#Convert string to numeric 
titanic_df['Sex'] = LabelEncoder().fit_transform(titanic_df['Sex']) 
titanic_df['Embarked'] = LabelEncoder().fit_transform(titanic_df['Embarked'])
titanic_df.info()

In [ ]:
#Heatmap of data 
sns.heatmap(titanic_df.corr(), annot=True, cmap=sns.cubehelix_palette(start=2,as_cmap=True))

From the heatmap it can be seen that the strongest indicator to surviving the disaster was Fare with a correlation of 0.26. Surprisingly, sex and class have a negative correlation despite both seeming to show that a female passenger or 1st class passenger was more likely to survive than a male or 3rd class passenger. 

In [ ]:
sns.boxplot(data=titanic_df, x = 'Sex', y = 'Fare', hue = 'Survived')

This graph shows the impact fare had on the survival for both sexes. The heatmap surprisingly did not show a strong correlation for Age or Sex. This graph could somewhat show why this might be. For both sexes a low fare more likely meant dying in the disaster. However, more male passengers that paid a higher fare survived. This effect is even more defined in female passengers. The 75% percentile of not suviving female passengers is roughly equal with the 50% percentile of female suvivors, showing that more women who paid a higher fare were more likely to survive. 

In [ ]:
sns.boxplot(data=titanic_df, x = 'Pclass', y = 'Fare', hue = 'Survived')

This graph comparing the class to fare is another indicator that fare had a big impact. There is not much variation in 2nd and 3rd class, but more 1st class passengers that paid a higher fare survived than 1st class passengers that paid a lower fare. 

#### In the movie, the upper-class passengers were given preference on lifeboats. Does this show in the data? 

Upperclass passengers would have stayed in 1st class and paid a higher fare. Comparing these to survival gives the following graphs:  

In [ ]:
sns.histplot(data=titanic_df, x = 'Pclass', hue = 'Survived', multiple = 'stack')

Passengers from every class died, but staying in 3rd class severly lowered your chances of survival, while staying in 1st class raised your chances of survival. In 2nd class these were roughly equal. As passengers on lifeboats would most likely have survived the icy waters, it seems likely that 1st class passengers were given preferential treatment compared to the other two classes. 

In [ ]:
sns.boxplot(data=titanic_df, x = 'Survived', y='Fare') 

In this graph the fare is compared to survival. The 50% quartile of 1 is on the same level as the 75% quartile of 0. The maximum of 0 is about the same as the 75% quartile of 1. That shows that more passengers that survived had also bought a more expensive ticket. This is a further indicator that upper-class passengers were given preferential treatment. 

#### “Women and children first”. Was this the case? 

To answer the question the previous graphs on Survival compared to Age, Sex and Parch should be considered. Additionally to the following graph:  

In [ ]:
sns.boxplot(data=titanic_df, x = 'Sex', y = 'Age', hue = 'Survived') 

As the minimum value for surviving male passengers is lower than for male passenger that died (not taking into account outliers), it seems to indicate that age played a role in a male passenger's chances of survival and that boys were more likely to be saved compared to older male passengers. Interestingly, the 25% and 75% quartiles of female passengers that died is lower than for surviving female passengers. This suggests that older female passengers were more likely to be saved than a female passenger in her early 20s. 

As shown in the previous graph of 'Sex vs. Survived' more female passengers survived than male passengers. Additionally, in the 'Age vs. Survived' graph passengers below the age of 10 were more likely to survive than passengers of any other age group. Another indicator is the 'Parch vs Survived' graph. Having one child or parent raised the chances of survival, possibly because mothers with children were prioritised. Altogether, this does seem to indicate that a policy of 'Women and children first' was in place/followed. 

#### Add one other observation that you have noted in the dataset. 

In [ ]:
sns.boxplot(data=titanic_df, x = 'Pclass', y = 'Age', hue = 'Survived')

Finally, this graph shows the age distribution in each passenger class. The majority of passengers in 1st class (surviving and not) were older than in 2nd or 3rd class. As you age you are more likely to earn more and so older people could afford the higher fare for 1st class. 

Children were passengers in all classes, however it is interesting to note that the minimum age for anyone who died in 2nd class is just below 20. This shows that all 2nd class children survived, whereas they are among the dead in all other classes (albeit anyone under 10 years old as outlier in 3rd class). 

It also shows that the passengers that died in 1st and 2nd class were generally older than those that survived and that the 25% quartile of survivors in 3rd class is younger than those that died. This suggests that older passengers helped younger passengers survive. 